In [1]:
import pandas as pd

stopsTimes = pd.read_csv('./data/stop_times.txt')

In [2]:
# lets get agencyId , stopID 
routes = pd.read_csv('./data/routes.txt')
routes = routes[['agency_id', 'route_id']]

In [3]:
trips = pd.read_csv('./data/trips.txt')

In [4]:
routesTrips = routes.merge(trips, how='left', left_on='route_id', right_on='route_id', suffixes=('_route', '_stop'))

In [5]:
routesTripsStopsSchedule = routesTrips.merge(stopsTimes, how='left', left_on='trip_id', right_on='trip_id')

In [6]:
agencyStops = routesTripsStopsSchedule[['agency_id', 'stop_id']].drop_duplicates()
agencyStops

,agency_id,stop_id
0,1,2010
1,1,2020
2,1,2030
3,1,2050
4,1,2060
...,...,...
1027121,1,17651
1027122,1,17680
1027137,1,76731
1027373,1,17040


In [7]:
stops = pd.read_csv('./data/stops.txt')
stops.shape

(6422, 13)

In [ ]:
# get all stops associated with any route

In [8]:
#  perform a api call to get the agency stops

# make a req to the API
import requests
def get_agency_stopsIds(agency_id):
    url = f'http://localhost:8081/onebusaway-api-webapp/api/where/stop-ids-for-agency/{agencyID}.json?key=test'
    response = requests.get(url)
    print("status code:", response.status_code)
    if response.status_code == 200:
        return response.json()
    else:
        return None

In [9]:
agencyID = '1'  
res = get_agency_stopsIds(agency_id=agencyID)
if res:
    stops = res['data']['list']
    print(len(stops), "stops found for agency", agencyID)
else:
    print("Failed to retrieve stops for agency", agencyID)

status code: 200
6422 stops found for agency 1


In [10]:
s1 = agencyStops[agencyStops['agency_id'] == 1].shape
s2 = agencyStops[agencyStops['agency_id'] == 23].shape
s3 = agencyStops[agencyStops['agency_id'] == 40].shape
print(f"Agency 1 stops: {s1[0]}")
print(f"Agency 23 stops: {s2[0]}")
print(f"Agency 40 stops: {s3[0]}")

print("Total stops in agencyStops DataFrame:", s1[0] + s2[0] + s3[0])
print("Total stops in stops DataFrame:", agencyStops["stop_id"].nunique())

Agency 1 stops: 6361
Agency 23 stops: 32
Agency 40 stops: 175
Total stops in agencyStops DataFrame: 6568
Total stops in stops DataFrame: 6422


In [11]:
agencyID = '23'  
res = get_agency_stopsIds(agency_id=agencyID)
if res:
    stops = res['data']['list']
    print(len(stops), "stops found for agency", agencyID)
else:
    print("Failed to retrieve stops for agency", agencyID)

status code: 200
0 stops found for agency 23


lets see the stops not for a1

In [12]:
stopsNon1 = agencyStops[agencyStops['agency_id'] != 1]

In [13]:
stops1 = agencyStops[agencyStops['agency_id'] == 1]
stops40 = agencyStops[agencyStops['agency_id'] == 40]
stops23 = agencyStops[agencyStops['agency_id'] == 23]

In [14]:
# get the unique stop_ids for agency 40 and 23 that are not in agency 1
unique_stops_40 = stops40[~stops40['stop_id'].isin(stops1['stop_id'])]
unique_stops_23 = stops23[~stops23['stop_id'].isin(stops1['stop_id'])]
print("Unique stops in agency 40 not in agency 1:", unique_stops_40['stop_id'].nunique())
print("Unique stops in agency 23 not in agency 1:", unique_stops_23['stop_id'].nunique())
# Check if the unique stops are actually unique
print("Unique stops in agency 40 not in agency 1:", unique_stops_40['stop_id'].unique())
print("Unique stops in agency 23 not in agency 1:", unique_stops_23['stop_id'].unique())
# Check if the unique stops are actually unique
print("Total unique stops in agency 40 not in agency 1:", unique_stops_40['stop_id'].nunique()) 


Unique stops in agency 40 not in agency 1: 40
Unique stops in agency 23 not in agency 1: 21
Unique stops in agency 40 not in agency 1: [77998 77090 38235 38567 77093 72487 71954 68496 72456 71341 71335 21850
 72458 71951 72488  1073  1215  1230 67960 68035 68042 67636 67652 67620
 67700 67720 11130  1920 64590 80485 64566 72329 64502 64555   730 62329
 57892 46476 68006 59911]
Unique stops in agency 23 not in agency 1: [26698 26701 26705 26680 26689 26693 26700 26702  1619  1630  1551  1652
  1662  1672  1682 11175 41988  1681  1671  1661  1651]
Total unique stops in agency 40 not in agency 1: 40


In [15]:
trips[trips["route_id"] == 100196]

,route_id,service_id,trip_id,trip_headsign,tts_trip_headsign,trip_short_name,direction_id,block_id,shape_id,peak_flag,fare_id,wheelchair_accessible,bikes_allowed
16985,100196,23519,701367268,Shoreline CC Mountlake Terrace Station,shoreline community college mountlake terrace ...,NaN,1,7633622,30331011,0,101,1,1
16986,100196,23519,701367458,Kenmore P&R Mountlake Terrace Station,kenmore park and ride mountlake terrace station,NaN,0,7633622,41331010,0,101,1,1
16987,100196,23519,701367608,Kenmore P&R Mountlake Terrace Station,kenmore park and ride mountlake terrace station,NaN,0,7633621,41331010,0,101,1,1
16988,100196,23519,701382398,Shoreline CC Mountlake Terrace Station,shoreline community college mountlake terrace ...,NaN,1,7633667,30331011,0,101,1,1
16989,100196,17361,701394448,Shoreline CC Mountlake Terrace Station,shoreline community college mountlake terrace ...,NaN,1,7633693,30331011,0,101,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29364,100196,86832,724954948,Kenmore P&R Mountlake Terrace Station,kenmore park and ride mountlake terrace station,NaN,0,7633833,41331010,0,101,1,1
29402,100196,86832,724969468,Kenmore P&R Mountlake Terrace Station,kenmore park and ride mountlake terrace station,NaN,0,7633769,41331010,0,101,1,1
29403,100196,86832,724969498,Kenmore P&R Mountlake Terrace Station,kenmore park and ride mountlake terrace station,NaN,0,7633897,41331010,0,101,1,1
29404,100196,86832,724969638,Shoreline CC Mountlake Terrace Station,shoreline community college mountlake terrace ...,NaN,1,7633897,30331011,0,101,1,1


In [16]:
routes[routes["route_id"] == 100196]

,agency_id,route_id
52,1,100196


In [17]:
import os
def get_stops_for_agency(agency_id: str, gtfs_path: str = 'data'):
    """
    Retrieves all unique stop_ids for a given agency_id from GTFS data.

    This function works by linking data from different GTFS files:
    1. It finds all routes associated with the agency_id in routes.txt.
    2. It finds all trips associated with those routes in trips.txt.
    3. It finds all stops associated with those trips in stop_times.txt.
    4. It returns a unique list of stop_ids.

    Args:
        agency_id (str): The ID of the agency to fetch stops for.
                         (e.g., 'DTA' from a sample agency.txt).
        gtfs_path (str): The path to the directory containing GTFS .txt files.
                         Defaults to 'data'.

    Returns:
        list[str]: A list of unique stop_id strings for the specified agency.
                   Returns an empty list if the agency or related data
                   cannot be found.
    """
    print(f"Searching for stops for agency '{agency_id}' in '{gtfs_path}'...")

    # Define paths to the required GTFS files
    routes_file = os.path.join(gtfs_path, 'routes.txt')
    trips_file = os.path.join(gtfs_path, 'trips.txt')
    stop_times_file = os.path.join(gtfs_path, 'stop_times.txt')

    required_files = [routes_file, trips_file, stop_times_file]

    # --- 1. Check if all necessary files exist ---
    for f in required_files:
        if not os.path.exists(f):
            print(f"Error: Required GTFS file not found at '{f}'")
            return []

    try:
        # --- 2. Load GTFS data into pandas DataFrames ---
        # Only load columns we absolutely need to save memory
        routes_df = pd.read_csv(routes_file, usecols=['agency_id', 'route_id'], dtype={'agency_id': str})
        trips_df = pd.read_csv(trips_file, usecols=['route_id', 'trip_id'])
        stop_times_df = pd.read_csv(stop_times_file, usecols=['trip_id', 'stop_id'])

        # --- 3. Filter routes for the specified agency ---
        agency_routes = routes_df[routes_df['agency_id'] == agency_id]

        if agency_routes.empty:
            print(f"Warning: No routes found for agency_id '{agency_id}'.")
            # You might want to check your agency.txt to ensure the ID is correct.
            # We can try to provide available agency IDs for convenience.
            try:
                agency_df = pd.read_csv(os.path.join(gtfs_path, 'agency.txt'), dtype={'agency_id': str})
                available_agencies = agency_df['agency_id'].unique().tolist()
                print(f"Available agency IDs are: {available_agencies}")
            except FileNotFoundError:
                print("Could not read agency.txt to suggest available IDs.")
            return []

        # --- 4. Merge dataframes to link agency -> routes -> trips -> stops ---
        # Merge agency's routes with all trips to find trips for this agency
        agency_trips = pd.merge(
            left=agency_routes,
            right=trips_df,
            on='route_id'
        )

        # Merge agency's trips with all stop times to find stops for this agency
        agency_stop_times = pd.merge(
            left=agency_trips,
            right=stop_times_df,
            on='trip_id'
        )

        # --- 5. Extract unique stop_ids and return as a list ---
        unique_stop_ids = agency_stop_times['stop_id'].unique().tolist()

        return unique_stop_ids

    except Exception as e:
        print(f"An error occurred while processing the GTFS files: {e}")
        return []

In [18]:
stopsByFunction = get_stops_for_agency(agency_id='1', gtfs_path='./data')
print(f"Number of unique stops for agency '1': {len(stopsByFunction)}")

Searching for stops for agency '1' in './data'...
Number of unique stops for agency '1': 6361


In [19]:
def get_stops_for_agency(agency_id: str, gtfs_path: str = 'data'):
    """
    Retrieves all unique stop_ids for a given agency_id from GTFS data.

    This function works by linking data from different GTFS files:
    1. It finds all routes associated with the agency_id in routes.txt.
    2. It finds all trips associated with those routes in trips.txt.
    3. It finds all stops associated with those trips in stop_times.txt.
    4. It returns a unique list of stop_ids.

    Args:
        agency_id (str): The ID of the agency to fetch stops for.
        gtfs_path (str): The path to the directory containing GTFS .txt files.

    Returns:
        list[str]: A list of unique stop_id strings for the specified agency.
    """
    print(f"Searching for stops for agency '{agency_id}' in '{gtfs_path}'...")
    # Define paths to the required GTFS files
    routes_file = os.path.join(gtfs_path, 'routes.txt')
    trips_file = os.path.join(gtfs_path, 'trips.txt')
    stop_times_file = os.path.join(gtfs_path, 'stop_times.txt')

    # Check for file existence
    for f in [routes_file, trips_file, stop_times_file]:
        if not os.path.exists(f):
            print(f"Error: Required GTFS file not found at '{f}'")
            return []

    try:
        # Load necessary data
        routes_df = pd.read_csv(routes_file, usecols=['agency_id', 'route_id'], dtype={'agency_id': str})
        trips_df = pd.read_csv(trips_file, usecols=['route_id', 'trip_id'])
        stop_times_df = pd.read_csv(stop_times_file, usecols=['trip_id', 'stop_id'])

        # Filter routes for the agency
        agency_routes = routes_df[routes_df['agency_id'] == agency_id]
        if agency_routes.empty:
            print(f"Warning: No routes found for agency_id '{agency_id}'.")
            return []

        # Merge to find all stops for the agency
        agency_trips = pd.merge(agency_routes, trips_df, on='route_id')
        agency_stop_times = pd.merge(agency_trips, stop_times_df, on='trip_id')
        
        return agency_stop_times['stop_id'].unique().tolist()

    except Exception as e:
        print(f"An error occurred while processing the GTFS files: {e}")
        return []

def get_stops_for_route(route_id: str, gtfs_path: str = 'data'):
    """
    Retrieves all unique stop_ids for a given route_id from GTFS data.

    Args:
        route_id (str): The ID of the route to fetch stops for.
        gtfs_path (str): The path to the directory containing GTFS .txt files.

    Returns:
        list[str]: A list of unique stop_id strings for the specified route.
    """
    print(f"Searching for stops for route '{route_id}' in '{gtfs_path}'...")
    trips_file = os.path.join(gtfs_path, 'trips.txt')
    stop_times_file = os.path.join(gtfs_path, 'stop_times.txt')

    for f in [trips_file, stop_times_file]:
        if not os.path.exists(f):
            print(f"Error: Required GTFS file not found at '{f}'")
            return []

    try:
        trips_df = pd.read_csv(trips_file, usecols=['route_id', 'trip_id'])
        stop_times_df = pd.read_csv(stop_times_file, usecols=['trip_id', 'stop_id'])

        # Filter trips for the given route
        route_trips = trips_df[trips_df['route_id'] == route_id]
        if route_trips.empty:
            print(f"Warning: No trips found for route_id '{route_id}'.")
            return []

        # Merge to find all stops for the route
        route_stop_times = pd.merge(route_trips, stop_times_df, on='trip_id')

        return route_stop_times['stop_id'].unique().tolist()

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

def get_stops_for_trip(trip_id: str, gtfs_path: str = 'data'):
    """
    Retrieves all stop_ids for a given trip_id from GTFS data.

    Args:
        trip_id (str): The ID of the trip to fetch stops for.
        gtfs_path (str): The path to the directory containing GTFS .txt files.

    Returns:
        list[str]: A list of stop_id strings for the specified trip.
    """
    print(f"Searching for stops for trip '{trip_id}' in '{gtfs_path}'...")
    stop_times_file = os.path.join(gtfs_path, 'stop_times.txt')

    if not os.path.exists(stop_times_file):
        print(f"Error: Required GTFS file not found at '{stop_times_file}'")
        return []

    try:
        stop_times_df = pd.read_csv(stop_times_file, usecols=['trip_id', 'stop_id', 'stop_sequence'])
        
        # Filter stops for the given trip and sort by sequence
        trip_stops = stop_times_df[stop_times_df['trip_id'] == trip_id].sort_values('stop_sequence')
        
        if trip_stops.empty:
            print(f"Warning: No stops found for trip_id '{trip_id}'.")
            return []

        return trip_stops['stop_id'].tolist()

    except Exception as e:
        print(f"An error occurred: {e}")
        return []


In [20]:
stopsFor100232Route = get_stops_for_route(route_id=100196, gtfs_path='./data')
print((77998) in stopsFor100232Route)
print(f"Number of unique stops for route '100232': {len(stopsFor100232Route)}")

Searching for stops for route '100196' in './data'...


False
Number of unique stops for route '100232': 85


In [21]:
import pandas as pd
import os
from typing import List, Dict, Optional

def get_stops_for_agency(data_folder: str, agency_id: str) -> List[Dict]:
    """
    Load GTFS data and return all stops served by a specific agency.
    
    Args:
        data_folder (str): Path to the folder containing GTFS files
        agency_id (str): The agency ID to filter stops for
        
    Returns:
        List[Dict]: List of stop dictionaries with stop information
    """
    
    # Define file paths
    stops_file = os.path.join(data_folder, 'stops.txt')
    stop_times_file = os.path.join(data_folder, 'stop_times.txt')
    trips_file = os.path.join(data_folder, 'trips.txt')
    routes_file = os.path.join(data_folder, 'routes.txt')
    
    # Check if required files exist
    required_files = [stops_file, stop_times_file, trips_file, routes_file]
    for file_path in required_files:
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Required GTFS file not found: {file_path}")
    
    try:
        # Load GTFS data
        print("Loading GTFS data...")
        stops_df = pd.read_csv(stops_file)
        stop_times_df = pd.read_csv(stop_times_file)
        trips_df = pd.read_csv(trips_file)
        routes_df = pd.read_csv(routes_file)
        
        print(f"Loaded {len(stops_df)} stops, {len(trips_df)} trips, {len(routes_df)} routes")
        
        # Filter routes by agency
        agency_routes = routes_df[routes_df['agency_id'] == agency_id]
        if agency_routes.empty:
            print(f"No routes found for agency_id: {agency_id}")
            return []
        
        print(f"Found {len(agency_routes)} routes for agency {agency_id}")
        
        # Get trips for agency routes
        agency_trips = trips_df[trips_df['route_id'].isin(agency_routes['route_id'])]
        print(f"Found {len(agency_trips)} trips for agency routes")
        
        # Get stop_times for agency trips
        agency_stop_times = stop_times_df[stop_times_df['trip_id'].isin(agency_trips['trip_id'])]
        print(f"Found {len(agency_stop_times)} stop_times for agency trips")
        
        # Get unique stops served by the agency
        agency_stop_ids = agency_stop_times['stop_id'].unique()
        agency_stops = stops_df[stops_df['stop_id'].isin(agency_stop_ids)]
        
        print(f"Found {len(agency_stops)} unique stops for agency {agency_id}")
        
        # Convert to list of dictionaries (similar to API response format)
        stops_list = []
        for _, stop in agency_stops.iterrows():
            stop_dict = {
                'stop_id': stop['stop_id'],
                'stop_name': stop['stop_name'],
                'stop_lat': float(stop['stop_lat']) if pd.notna(stop['stop_lat']) else None,
                'stop_lon': float(stop['stop_lon']) if pd.notna(stop['stop_lon']) else None,
                'stop_code': stop.get('stop_code', ''),
                'stop_desc': stop.get('stop_desc', ''),
                'zone_id': stop.get('zone_id', ''),
                'stop_url': stop.get('stop_url', ''),
                'location_type': int(stop.get('location_type', 0)) if pd.notna(stop.get('location_type')) else 0,
                'parent_station': stop.get('parent_station', ''),
                'wheelchair_boarding': int(stop.get('wheelchair_boarding', 0)) if pd.notna(stop.get('wheelchair_boarding')) else 0
            }
            stops_list.append(stop_dict)
        
        # Sort by stop_id for consistent ordering
        stops_list.sort(key=lambda x: x['stop_id'])
        
        return stops_list
        
    except Exception as e:
        print(f"Error processing GTFS data: {str(e)}")
        raise

def compare_with_api(gtfs_stops: List[Dict], api_stops: List[Dict]) -> Dict:
    """
    Compare GTFS-derived stops with API response stops.
    
    Args:
        gtfs_stops: List of stops from GTFS data
        api_stops: List of stops from API response
        
    Returns:
        Dict: Comparison results
    """
    gtfs_stop_ids = {stop['stop_id'] for stop in gtfs_stops}
    api_stop_ids = {stop['stop_id'] for stop in api_stops}
    
    comparison = {
        'gtfs_count': len(gtfs_stops),
        'api_count': len(api_stops),
        'common_stops': len(gtfs_stop_ids & api_stop_ids),
        'gtfs_only': gtfs_stop_ids - api_stop_ids,
        'api_only': api_stop_ids - gtfs_stop_ids,
        'match_percentage': (len(gtfs_stop_ids & api_stop_ids) / max(len(gtfs_stop_ids), len(api_stop_ids))) * 100 if gtfs_stop_ids or api_stop_ids else 0
    }
    
    return comparison


In [22]:
li = get_stops_for_agency(data_folder='./data', agency_id=1)
print(f"Number of stops for agency '1': {len(li)}")

Loading GTFS data...
Loaded 6422 stops, 29812 trips, 145 routes
Found 136 routes for agency 1
Found 27609 trips for agency routes
Found 1006714 stop_times for agency trips
Found 6361 unique stops for agency 1
Number of stops for agency '1': 6361
